In [1]:
from osgeo import gdal

In [2]:
import os
from utils.GenUtils import make_folder, get_paths, chunk_creator, folder_file_size, question

In [3]:
from pysis import Isis

/usr/local/lib/python3.8/dist-packages/pysis/env.py:33: RuntimeWarning: Could not find isis. Is `ISISROOT` set?
  warnings.warn('Could not find isis. Is `ISISROOT` set?', RuntimeWarning)


In [4]:
isis = Isis(path=os.environ['ISISROOT'])

In [5]:
def cub2map(src,maptemplate):
    
    src_basename = os.path.basename(src).split('.IMG')[0]
    dst_basename = DPATH+'/'+src_basename
    dst_basefpath = DPATH+'/'+src_basename
    print(dst_basename)
    if os.path.isfile(dst_basename+'_cal_map.JP2'):
        print ("File exist")
    elif os.path.isfile(dst_basename+'.JP2'):
        print ("File exist")
    else:
        dst = dst_basefpath+'.cub'
        print('Creating cube')
        isis.lronac2isis(from_=src, to=dst)
        init = None
        while init == None:
            try:
                isis.spiceinit(from_=dst, web='yes')
                init = 'Done'
            except:
                pass
        dst_cal = dst_basefpath+'_cal.cub'
        print('Calibrating Cube')
        isis.lronaccal(from_=dst,to=dst_cal)
        
        dst_map = dst_cal.split('.cub')[0]+'_map.cub'
        print('Projecting cube to map')
        try:
            print('Mapping')
            isis.cam2map(from_=dst_cal, to=dst_map, map_=maptemplate)
            print('done')
            src_map = gdal.Open(dst_map)
            # format = 'GTiff'
            # driver = gdal.GetDriverByName(format)
            dst_jp2 = dst_basefpath+'.JP2'
            # isis2std(from_=dst_map, to=dst_gtiff,mode='grayscale',format='TIFF',bittype='8BIT',stretch='linear', minpercent=0.2, maxpercent=99.8)
            opts = f'-a_nodata none -mask none -scale -ot Byte'
            # print('Exporting map')
            gdal.Translate(dst_jp2,src_map,options=opts)
            try:
                os.remove(dst)
            except Exception as e:
                print(e)
            try:
                os.remove(dst_cal)
            except Exception as e:
                print(e)
            try:
                os.remove(dst_map)
            except Exception as e:
                print(e)
        except Exception as e:
            print(src_basename, e)
            pass

In [6]:
def parallel_cub2map(files, JOBS,maptemplate):
    from joblib import Parallel, delayed
    Parallel (n_jobs=JOBS)(delayed(cub2map)(files[i],maptemplate)
                            for i in range(len(files)))

In [7]:
def main(PATH, DPATH, ixt,maptemplate):
    image_list = get_paths(PATH, ixt) 
    total_size, max_size, av_fsize = folder_file_size(PATH,image_list)
    from tqdm import tqdm
    import psutil
    avram=psutil.virtual_memory().total >> 30
    avcores=psutil.cpu_count(logical=False)
    avthreads=psutil.cpu_count(logical=True)
    ram_thread = avram/avthreads
    req_mem = avthreads*max_size
    if req_mem > avcores and req_mem > avram:
        JOBS = avcores
    else:
        JOBS = avcores
    with tqdm(total=len(image_list),
             desc = 'Generating Images',
             unit='File') as pbar:
        filerange = len(image_list)
        chunksize = round(filerange/JOBS)
        if chunksize <1:
            chunksize=1
            JOBS = filerange
        chunks = []
        for c in chunk_creator(image_list, JOBS):
            chunks.append(c)
        # from time import time
        from datetime import datetime
        for i in range(len(chunks)):
            start = datetime.now()
            dt_string = start.strftime("%d/%m/%Y %H:%M:%S")
            print(f'Loop {i} started at: {dt_string}', chunks[i])
            files = chunks[i]
            parallel_cub2map(files, JOBS,maptemplate)
            pbar.update(JOBS)

In [ ]:
if __name__ == "__main__":
    PATH = './'
    DPATH = PATH+'/processed'
    os.makedirs(DPATH, exist_ok=True)
    ixt = 'IMG'
    maptemplate = './utils/maptemplate/map_template_moon_eqc.map'
    main(PATH, DPATH, ixt,maptemplate)

Generating Images:   0%|                                                                                                                                                                                                                                                                                                                                 | 0/1 [00:00<?, ?File/s]

Loop 0 started at: 02/11/2021 09:45:48 ('.//M1346290850RE.IMG',)
.//processed/M1346290850RE
Creating cube
Calibrating Cube
Projectin cube to map
Mapping
